In [1]:
import cv2
import numpy as np

img = cv2.imread('data/lena.png')
#cv2.imshow('',img)
#cv2.waitKey(0)

In [14]:
img_line = img.copy()
img_line = cv2.line(img_line,(60,40),(402,510),(0,0,255),2)
img_line = cv2.putText(img_line,'line', (60, 40), 0, 1, (255, 0,0), 2)
cv2.imshow('line',img_line)
cv2.waitKey(0)

-1

In [2]:
img_rect = img.copy()
img_rect = cv2.rectangle(img_rect, (60, 40), (420, 510), (0, 0, 255), 3)
#cv2.imshow('rect',img_rect)
#cv2.waitKey(0)

## 解法二


如果希望得知矩型邊框的位置

顏色的操作 (對明亮度做直方圖均衡)

鏡像可以透過四則運算得知

透過建構 transformation matrix 做縮放

把矩型邊框的點與 transformation matrix 相乘就會得到縮放後的位置

畫圖
得到的圖的結果正確，同時也知道新的矩型邊框座標點

In [18]:
img_hw = img_rect.copy()

#img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
#img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
#img_hw = cv2.cvtColor(img_hw,cv2.COLOR_HSV2BGR)


img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HLS)
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
img_hw = cv2.cvtColor(img_hw,cv2.COLOR_HLS2BGR)
img_hw[img_hw[..., -1] <255] = 0 

#print(img_hw)
#cv2.imshow('rect',img_hw)
#cv2.waitKey(0)
list=[]
for y in range(img_hw.shape[0]):#y
        for x in range(img_hw.shape[1]):#x
            (b,g,r) = img_hw[y,x]                        
            if ( r == 255):
                list.append([y,x])
max_=max(list)               
min_=min(list)
print("max=",max_)
print("min=",min_)
            #    print(img_hw.item() )
#print(img_hw.shape[2])
#for x in range(0,img_hw.shape[0]):
#    for y in range(0,img_hw.shape[1]):
#        if img_hw[x,y] == [1, 1, 255]: 
#            print(img_hw[x,y])

max= [511, 421]
min= [38, 60]


In [11]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]
h, w = img_hw.shape[:2]#y,x
center = ( w/2, h/2)
print("center=",center)
# 水平鏡像
img_hw = cv2.flip(img_hw,1)

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
print(point1[0])
point1[0] = w - point1[0]
point2[0] = w - point2[0]
print("org-->point1={0},point2={1}".format(point1,point2))
point1[0] ,point2[0] = point2[0],point1[0]
print("new-->point1={0},point2={1}".format(point1,point2))

center= (256.0, 256.0)
60
org-->point1=[452, 40],point2=[92, 510]
new-->point1=[92, 40],point2=[452, 510]


In [5]:
"""
縮放處理 (0.5 倍)
"""
fx = 0.5
fy = 0.5
resize_col = int(img_hw.shape[1]*fx)
resize_row = int(img_hw.shape[0]*fy)

# 建構 scale matrix
#[a,c,e]-->a,d=scale; c,b=旋轉角度 , e,f=移動
#[b,d,f]
M_scale = np.array([[fx,0,0],[0,fx,0]])
M_scale = M_scale.astype('float32')
img_hw = cv2.warpAffine(img_hw,M_scale,(resize_row,resize_col))

In [6]:
# 把左上跟右下轉為矩陣型式
bbox = np.array((point1, point2), dtype=np.float32)
print('M_scale.shape={}, bbox.shape={}'.format(M_scale.shape, bbox.shape))

# 做矩陣乘法可以使用 `np.dot`, 為了做矩陣乘法, M_scale 需要做轉置之後才能相乘
homo_coor_result = np.dot(M_scale.T, bbox)
homo_coor_result = homo_coor_result.astype('uint8')
#print(homo_coor_result)
scale_point1 = tuple(homo_coor_result[0])
scale_point2 = tuple(homo_coor_result[1])
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

# 畫圖
cv2.rectangle(img_hw, scale_point1, scale_point2, (0, 0, 255), 3)
cv2.imshow('image', img_hw)
cv2.waitKey(0)
cv2.destroyAllWindows()


M_scale.shape=(2, 3), bbox.shape=(2, 2)
origin point1=[92, 40], origin point2=[452, 510]
resize point1=(46, 20), resize point2=(226, 255)
